Installer des versions spécifiques de PyTorch (2.1.0) et TorchVision (0.12.0) pour garantir leur compatibilité.

In [ ]:
!pip install torch==2.1.0
!pip torchvision==0.12.0

Afficher la version de TorchVision installée en important la bibliothèque et en imprimant sa version.

In [ ]:
import torchvision
print(torchvision.__version__)

Importer les bibliothèques nécessaires pour manipuler les fichiers, effectuer des calculs numériques, créer et entraîner des réseaux de neurones, gérer des ensembles de données, transformer des images, et afficher une barre de progression.

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
from tqdm.notebook import tqdm

La fonction load_images permet Charger des images à partir d'un dossier donné, les redimensionne, les normalise entre -1 et 1, et les convertit en tenseurs PyTorch au format (C, H, W) avant de les retourner sous forme de liste.

In [2]:
# ----------- Fonction de chargement des images -----------
def load_images(data_path, img_shape):
    images = []
    for img_name in os.listdir(data_path):
        img_path = os.path.join(data_path, img_name)
        img = Image.open(img_path).convert("RGB")
        img = img.resize(img_shape)
        img = np.array(img) / 127.5 - 1  # Normalisation entre -1 et 1
        img = torch.from_numpy(img).permute(2, 0, 1).float()  # Changer l'ordre pour (C, H, W)
        images.append(img)
    return images

Créer un dataset personnalisé pour charger et gérer des images, où chaque image est prétraitée (redimensionnée et normalisée) via la fonction load_images, avec un accès facile à leur longueur totale et à une image spécifique par son index.

In [3]:
# ----------- Dataset personnalisé -----------
class CustomImageDataset(Dataset):
    def __init__(self, data_path, img_shape):
        self.images = load_images(data_path, img_shape)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx]

Définir un générateur de réseau de neurones pour créer des images à partir d'un vecteur latent, en utilisant des couches entièrement connectées et transposées convolutives, suivi de normalisations et d'activations pour produire une image avec des valeurs normalisées entre -1 et 1.

In [4]:
# ----------- Générateur -----------
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128 * 32 * 32),
            nn.LeakyReLU(0.2),
            nn.Unflatten(1, (128, 32, 32)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.4),# 0.2
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

Définir un discriminateur, un réseau de neurones convolutif conçu pour différencier les images réelles des images générées, en utilisant des couches convolutives, des normalisations, des activations et une sortie sigmoïde pour produire une probabilité.

In [5]:
# ----------- Discriminateur -----------
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.4), #0.2
            nn.Flatten(),
            nn.Linear(128 * 32 * 32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

Initialiser les paramètres du modèle et les hyperparamètres : dimension du vecteur latent (100), taille des images (128x128), chemin des données, taille de batch (64), nombre d'époques (2000), taux d'apprentissage (0.0002), et sélection de l'appareil (GPU si disponible, sinon CPU).

In [6]:
# ----------- Initialisation et hyperparamètres -----------
latent_dim = 100
img_shape = (128, 128)
data_path = "Dataset/tumor"
batch_size = 64
num_epochs = 2000
lr = 0.0002
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Charger les données avec un DataLoader, crée les modèles de générateur et de discriminateur, initialise les optimiseurs avec l'algorithme Adam et définit la fonction de perte (BCELoss) pour l'entraînement du GAN.

In [7]:
# Charger les données
dataset = CustomImageDataset(data_path, img_shape)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Créer les modèles
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Optimiseurs
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

# Fonction de perte
criterion = nn.BCELoss()

Entraîner un GAN pendant 2000 époques en optimisant le générateur et le discriminateur. À chaque époque, le générateur crée de fausses images et le discriminateur apprend à distinguer les vraies des fausses. Les pertes des deux modèles sont calculées et les paramètres sont mis à jour. Toutes les 100 époques, des images générées sont sauvegardées.

In [ ]:
# ----------- Entraînement -----------
for epoch in tqdm(range(num_epochs), desc="Training GAN"):
# for epoch in range(num_epochs):
    for i, real_images in enumerate(data_loader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        
        # Vrais et faux labels
        valid = torch.ones((batch_size, 1), device=device)
        fake = torch.zeros((batch_size, 1), device=device)
        
        # -------- Entraînement du générateur --------
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, latent_dim, device=device)
        generated_images = generator(z)
        g_loss = criterion(discriminator(generated_images), valid)
        g_loss.backward()
        optimizer_G.step()
        
        # -------- Entraînement du discriminateur --------
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(real_images), valid)
        fake_loss = criterion(discriminator(generated_images.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        
    print(f"Epoch {epoch+1}/{num_epochs} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")
    
    # Sauvegarde des images toutes les 100 époques
    if epoch % 100 == 0:
        with torch.no_grad():
            for idx, img in enumerate(generated_images):
                save_image(img, f"torch_generated_image_epoch_{epoch}_img_{idx}.png", normalize=True)
#         with torch.no_grad():
#             generated_images = generator(z).detach().cpu()
#             save_image(generated_images, f"torch_generated_images_epoch_{epoch}.png", normalize=True)

Training GAN:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 1/2000 | D Loss: 0.0030 | G Loss: 5.3677
Epoch 2/2000 | D Loss: 0.0043 | G Loss: 5.3135
Epoch 3/2000 | D Loss: 0.0026 | G Loss: 5.6288
Epoch 4/2000 | D Loss: 0.0016 | G Loss: 5.9461
Epoch 5/2000 | D Loss: 0.0018 | G Loss: 6.0075
Epoch 6/2000 | D Loss: 0.0032 | G Loss: 5.8974
Epoch 7/2000 | D Loss: 0.0011 | G Loss: 6.6582
Epoch 8/2000 | D Loss: 0.0041 | G Loss: 6.4577
Epoch 9/2000 | D Loss: 0.0053 | G Loss: 5.5889
Epoch 10/2000 | D Loss: 1.4655 | G Loss: 9.6110
Epoch 11/2000 | D Loss: 0.0893 | G Loss: 8.1159
Epoch 12/2000 | D Loss: 2.4654 | G Loss: 25.9388
Epoch 13/2000 | D Loss: 0.8936 | G Loss: 6.4326
Epoch 14/2000 | D Loss: 0.7343 | G Loss: 16.1921
Epoch 15/2000 | D Loss: 0.0000 | G Loss: 14.9037
Epoch 16/2000 | D Loss: 0.3706 | G Loss: 20.4235
Epoch 17/2000 | D Loss: 0.0105 | G Loss: 7.5242
Epoch 18/2000 | D Loss: 0.8603 | G Loss: 6.5126
Epoch 19/2000 | D Loss: 0.0576 | G Loss: 4.8867
Epoch 20/2000 | D Loss: 0.0270 | G Loss: 4.1122
Epoch 21/2000 | D Loss: 0.0515 | G Loss: 5.09

Enfin, après l'entraînement, le générateur produit et sauvegarde 16 nouvelles images.

In [ ]:
# ----------- Générer des images après l'entraînement -----------
z = torch.randn(16, latent_dim, device=device)
generated_images = generator(z)
with torch.no_grad():
    for idx, img in enumerate(generated_images):
        save_image(img, f"torch_final_generated_image_{idx}.png", normalize=True)
# z = torch.randn(16, latent_dim, device=device)
# generated_images = generator(z)
# save_image(generated_images, "torch_final_generated_images.png", normalize=True)

Sauvegarder les poids du modèle du générateur et du discriminateur dans des fichiers .pth afin de pouvoir les recharger et les réutiliser plus tard.

In [ ]:
torch.save(generator.state_dict(), f"generator_model.pth")
torch.save(discriminator.state_dict(), f"discriminator_model.pth")

Charger les modèles préalablement sauvegardés en réinitialisant les instances du générateur et du discriminateur, puis charge les poids des modèles sauvegardés. Les modèles sont ensuite mis en mode évaluation pour désactiver certaines opérations comme la normalisation par lot pendant l'inférence.

In [ ]:
# Load models
# Créer des instances des modèles
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Charger les états sauvegardés
generator.load_state_dict(torch.load("generator_model.pth"))
discriminator.load_state_dict(torch.load("discriminator_model.pth"))

# Mettre en mode évaluation
generator.eval()
discriminator.eval()

Génèrer 408 images à partir de vecteurs aléatoires en utilisant le générateur, et les sauvegarde sous forme de fichiers .jpg dans le dossier "FacticeImage", avec un nom de fichier qui inclut l'index de l'image. Les images sont générées sans calcul de gradients pour économiser de la mémoire.

In [ ]:
for i in range(408):
    with torch.no_grad():
        z = torch.randn(1, latent_dim, device=device)  # Génère 1 vecteurs de bruit
        generated_images = generator(z).detach().cpu()
        save_image(generated_images, f"FacticeImage/tumor ({i}).jpg", normalize=True)
# with torch.no_grad():
#     z = torch.randn(16, latent_dim, device=device)  # Génère 16 vecteurs de bruit
#     generated_images = generator(z).detach().cpu()
#     save_image(generated_images, "tumor.jpg", normalize=True)